# Set up `rxmx.walker.Walker`

In [1]:
import lzma
from pathlib import Path

import dill as pickle
import numpy as np
from tqdm import tqdm

In [2]:
from scipy import stats
from scipy.optimize import minimize

In [3]:
import elm

Using database version X4-2024-12-31 located in: /mnt/home/beyerkyl/x4db/unpack_exfor-2024/X4-2024-12-31


In [4]:
import rxmc

In [5]:
evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")
likelihood_model_dir = Path("../likelihood_model/")
prior_path = Path("../prior/")

## Prior

In [6]:
with open(prior_path / "prior_distribution_chuq.pkl", "rb") as f:
    prior = pickle.load(f)

## read in observations

In [7]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

## read in physical models

In [8]:
with open(model_dir / "chuq_nn_model.pkl", "rb") as f:
    nn_model = pickle.load(f)

In [9]:
with open(model_dir / "chuq_pp_model.pkl", "rb") as f:
    pp_model = pickle.load(f)

## read in likelihood models

In [10]:
with open(likelihood_model_dir / "likelihood_model.pkl", "rb") as f:
    likelihood_model = pickle.load(f)

## Set up Evidence object

In [11]:
pp_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=pp_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in pp_elastic_obs.items()
]
nn_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=nn_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in nn_elastic_obs.items()
]

In [12]:
evidence = rxmc.evidence.Evidence(constraints=nn_constraints + pp_constraints)

In [13]:
starting_loc = prior.mean
for x, p in zip(starting_loc, elm.chuq.params):
    print(f"{p.name}: {x:1.2f}")

V0: 56.19
Ve: -0.36
Vt: 13.82
r0: 1.20
r0_0: -0.20
a0: 0.73
Wv0: 9.92
Wve0: 33.15
Wvew: 24.00
rw: 1.32
rw_0: -0.41
aw: 0.69
Ws0: 10.59
Wst: 27.09
Wse0: 20.00
Wsew: 36.38
Vso: 5.58
rso: 1.29
rso_0: -1.12
aso: 0.61
rc: 1.25
rc_0: 0.13


In [14]:
scale_factors = [0.001, 0.01, 0.1]

In [16]:
prior_stddev = np.sqrt(np.diag(prior.cov))
prop_stddevs = [prior_stddev * x for x in scale_factors]
prop_cov = [np.diag((sigma) ** 2) for sigma in prop_stddevs]

In [19]:
prop_stddevs[2]

array([0.1625, 0.0025, 0.614 , 0.003 , 0.0125, 0.0025, 0.3775, 2.2425,
       1.042 , 0.008 , 0.034 , 0.005 , 0.369 , 1.05  , 2.126 , 1.871 ,
       0.055 , 0.011 , 0.048 , 0.004 , 0.012 , 0.0105])

In [21]:
sampling_configs = [
    rxmc.param_sampling.SamplingConfig(
        starting_location=starting_loc,
        params=elm.chuq.params,
        proposal=rxmc.proposal.NormalProposalDistribution(
            cov=cov,
        ),
        prior=prior,
    )
    for cov in prop_cov
]

In [16]:
rng = np.random.default_rng(13)

In [23]:
walkers = [
    rxmc.walker.Walker(
        model_sample_conf=sampling_config,
        evidence=evidence,
        rng=np.random.default_rng(42),
    )
    for sampling_config in sampling_configs
]

In [25]:
for walker, s in zip(walkers, scale_factors):
    with lzma.open(f"./walker_chuq_prop_cov_scale_factor_{s}.pkl.xz", "wb") as f:
        pickle.dump(walker, f)

In [26]:
%%time
walkers[0].walk(100)

Batch: 1/1 completed, 100 steps. 
  Model parameter acceptance fraction: 0.520
CPU times: user 1min 6s, sys: 282 ms, total: 1min 6s
Wall time: 56.3 s


0.52

In [27]:
walkers[1].walk(100)

Batch: 1/1 completed, 100 steps. 
  Model parameter acceptance fraction: 0.490


0.49

In [28]:
walkers[2].walk(100)

Batch: 1/1 completed, 100 steps. 
  Model parameter acceptance fraction: 0.140


0.14